In [68]:
import numpy as np
import pandas as pd
import os
import math
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from operator import itemgetter

from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier


In [69]:
mutationRate = 0.001
crossOverRate = 0.06
iterations = 10
poolSize = 50

In [130]:
#Defining functions for genetic algorithms

def roulette(fitnessArray):
    index = 0
    cumalativeFitness = 0.0
    r = random.random()
    
    for i in range(len(fitnessArray)):
        cumalativeFitness += fitnessArray[i]
        if cumalativeFitness > r:
            return i


def selectFittest(fitness, rankedPool):
    while True:
        idx1 = roulette(fitness)
        idx2 = roulette(fitness)
        
        if idx1 is None or idx2 is None:
            continue
        elif idx1==idx2:
            continue
        else:
            break
    
    return rankedPool[idx1], rankedPool[idx2]

def crossover(chromosome1, chromosome2):
    randomSplitPoint = random.randint(1, len(chromosome1))
    print randomSplitPoint,chromosome1,chromosome2
    return np.concatenate(chromosome1[:randomSplitPoint]+chromosome2[randomSplitPoint:]), np.concatenate(chromosome2[:randomSplitPoint]+chromosome1[randomSplitPoint:])


def mutate(chromosome):

    mutatedChrom = []
    for ch in chromosome:
        if random.random()<mutationRate:
            if ch==1:
                mutatedChrom.append(0)
            else:
                mutatedChrom.append(1)
        else:
            mutatedChrom.append(ch)
    return mutatedChrom
    
def breed(chrome1, chrome2):
    if random.random()<crossOverRate:
        newChrome1, newChrome2 = crossover(chrome1, chrome2)
    else:
        newChrome1 = chrome1
        newChrome2 = chrome2
        
    newChrome1 = mutate(newChrome1)
    newChrome2 = mutate(newChrome2)
    
    return newChrome1, newChrome2

In [131]:
def rankPop(pool, X, y, classifier,fitnessFunction):
    scores = []
    for chromosome in pool:
        classifier = RandomForestClassifier()
        chosen_idx = [idx for gene, idx in zip(chromosome, range(X.shape[1])) if gene==1]
        if len(chosen_idx)==0:
            continue
        chosenX = X.iloc[:, chosen_idx]
        #performing leave-one out validation for instances less than 100
        #and 10 fold validation for others
        npoints = X.shape[0]
   
        if npoints <= 100:
            kf = KFold(n_splits = npoints)
        else:
            kf = KFold(n_splits = 10)
        
        kf.get_n_splits(X)
        classifier.fit(chosenX, y)
        train_X = []
        train_Y  = []
        prediction   = []
        predict_prob = []
        chosenX = np.array(chosenX)
        Y = np.array(y)
        for train_index, test_index in kf.split(X):
            classifier = RandomForestClassifier()
            for i in train_index:
                train_X.append(chosenX[i])
                train_Y.append(Y[i])

            classifier.fit(train_X, train_Y)
            for j in test_index:
                prediction.append(classifier.predict([chosenX[j]])[0])
                predict_prob.append(classifier.predict_proba([chosenX[j]])[0][1])
            train_X  = []
            train_Y  = []
        
   
        if(fitnessFunction == 'f-measure'):
            scores.append(f1_score(y_true=y,y_pred=prediction))
        elif(fitnessFunction == 'g-mean'):
            gScore = math.sqrt(precision_score(y_true = y, y_pred=prediction)*recall_score(y_true = y, y_pred =prediction ))
            scores.append(gscore)
        elif(fitnessFunction == 'accuracy'):
            scores.append(accuracy_score(y_true = y, y_pred = prediction))
        
    fitness = [x/sum(scores) for x in scores]
    pairedPop = zip(pool, fitness)
    rankedPop = sorted(pairedPop, key=itemgetter(-1), reverse = True)
    
    return rankedPop

In [132]:
def iteratePop(rankedPop):
    fitness = [item[-1] for item in rankedPop]
    rankedPool = [item[0] for item in rankedPop]
   
    new_pool = []
    new_pool.extend(rankedPool[:poolSize/15])
    
    while(len(new_pool)<poolSize):
        ch1, ch2 = selectFittest(fitness, rankedPool)
        ch1, ch2 = breed(ch1, ch2)
        
        new_pool.append(ch1)
        new_pool.append(ch2)
    
    return new_pool[:poolSize]

In [133]:
def geneticAlgoFit(datafile):
    datafile = pd.read_csv(datafile, dtype={'buggy':np.bool})
    X     = datafile.iloc[ : , :-1]
    y = datafile['buggy']

    pool = np.random.randint(0, 2, (poolSize, X.shape[1]))  
    for iteration in range(iterations):
        classifier = RandomForestClassifier()
        rankedPop = rankPop(pool, X, y, classifier,'accuracy')
#         print rankedPop
        pool = []
        pool = iteratePop(rankedPop)
        
    best_chromosome = rankPop(pool, X, y, classifier)[0][0]
    return best_chromosome

In [ ]:
datafile = 'dataset/dataset/camel-1.6.csv'
print geneticAlgoFit(datafile)

In [ ]:
directory = 'dataset/dataset'
for projectName in os.listdir(directory):
    print projectName
    print geneticAlgoFit(datafile)
